In [26]:
from typing import List, Literal, Union
import pandas as pd
import pm4py
from pm4py.objects.ocel.obj import OCEL
from pydantic.main import BaseModel
ocel = pm4py.read_ocel2_sqlite("/home/grkmr/Projects/ocelescope/src/backend/data/event_logs/ContainerLogistics-v2.sqlite")



/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_23_foreign_key_event_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/util/dataframe_utils.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pandas_utils.dataframe_column_string_to_datetime(df[col], format=timest_format, utc=True)
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/u

In [27]:
def getO2OWithTypes(ocel):
    o2o_with_types = pd.merge(ocel.o2o, ocel.objects[[ocel.object_id_column, ocel.object_type_column]], how="left")
    o2o_with_types = pd.merge(o2o_with_types, ocel.objects[[ocel.object_id_column, ocel.object_type_column]], how="left", left_on=f"{ocel.object_id_column}_2" , right_on=ocel.object_id_column, suffixes=["", "_new"])
    o2o_with_types = o2o_with_types[[ocel.object_id_column,f"{ocel.object_id_column}_2", ocel.qualifier, ocel.object_type_column, f"{ocel.object_type_column}_new"]]
    rename_map = {
        ocel.object_id_column: "source",
        f"{ocel.object_id_column}_2": "target",
        ocel.qualifier: "qualifier",
        ocel.object_type_column: "source_type",
        f"{ocel.object_type_column}_new": "target_type"
    }
    return o2o_with_types.rename(columns=rename_map)

In [28]:
getO2OWithTypes(ocel)

,source,target,qualifier,source_type,target_type
0,co1,td6,TD for CO,Customer Order,Transport Document
1,co2,td1,TD for CO,Customer Order,Transport Document
2,co3,td7,TD for CO,Customer Order,Transport Document
3,co4,td9,TD for CO,Customer Order,Transport Document
4,co5,td5,TD for CO,Customer Order,Transport Document
...,...,...,...,...,...
3365,cr2011,td590,CR for TD,Container,Transport Document
3366,cr1999,td586,CR for TD,Container,Transport Document
3367,cr1989,td583,CR for TD,Container,Transport Document
3368,cr1993,td584,CR for TD,Container,Transport Document


In [57]:
config = {
    "source":"Container",
    "target":"Transport Document",
    "min": 0,
    "max": 0,
    "mode":"include"
    
}

target = "target"
target_type_column = f"{target}_type"

o2o = getO2OWithTypes(ocel)
mask = (o2o["source_type"] == config["source"]) & (o2o["target_type"] == config["target"])

filtered_o2o = o2o[mask]
entity_counts = filtered_o2o.groupby(target).size().reset_index(name="entity_count")


if config["max"] is not None:
    entity_counts = entity_counts[(entity_counts["entity_count"] >= config["min"])& (entity_counts["entity_count"] <= config["max"])]
else:
    entity_counts = entity_counts[entity_counts["entity_count"] >= config["min"]]

is_not_target_type = ocel.objects[ocel.object_type_column] != (config[target])

is_in_filtered_ids = ocel.objects[ocel.object_id_column].isin(entity_counts[target])

if config["mode"] == "exclude":
    is_in_filtered_ids = ~is_in_filtered_ids

final_mask = is_not_target_type | is_in_filtered_ids
print(entity_counts)

Empty DataFrame
Columns: [target, entity_count]
Index: []


In [58]:
o2o = getO2OWithTypes(ocel)

In [60]:
o2o[ (o2o["source_type"] == config["source"]) & (o2o["target_type"] == config["target"])]

,source,target,qualifier,source_type,target_type
1371,cr2,td1,CR for TD,Container,Transport Document
1372,cr3,td1,CR for TD,Container,Transport Document
1373,cr4,td1,CR for TD,Container,Transport Document
1374,cr7,td2,CR for TD,Container,Transport Document
1375,cr14,td4,CR for TD,Container,Transport Document
...,...,...,...,...,...
3365,cr2011,td590,CR for TD,Container,Transport Document
3366,cr1999,td586,CR for TD,Container,Transport Document
3367,cr1989,td583,CR for TD,Container,Transport Document
3368,cr1993,td584,CR for TD,Container,Transport Document


In [ ]:
grouped_relations

In [91]:
from typing import Literal, Optional, cast, List
from pydantic import BaseModel
import pandas as pd

class RelationCountSummary(BaseModel):
    qualifier: str
    source: str
    target: str
    min_count: int
    max_count: int
    sum: int

def summarize_relation_counts(
    relation_table: pd.DataFrame,
    qualifier_col: str,
    source_type_col: str,
    target_type_col: str,
    source_id_col: str,
    target_id_col: str,
    direction: Literal["source", "target"]
) -> List[RelationCountSummary]:
    group_key = source_id_col if direction == "source" else target_id_col

    grouped_relations = (
        relation_table.groupby([group_key, qualifier_col, source_type_col, target_type_col])
        .size()
        .reset_index()
        .rename(columns={0: "count"})
    )

    summary = (
        grouped_relations.groupby([qualifier_col, source_type_col, target_type_col])["count"]
        .agg(["min", "max", "sum"])
        .reset_index()
        .rename(columns={"min": "min_count", "max": "max_count"})
    )

    summaries = [
        RelationCountSummary(
            qualifier=cast(str, row[qualifier_col]),
            source=cast(str, row[source_type_col]),
            target=cast(str, row[target_type_col]),
            min_count=cast(int, row["min_count"]),
            max_count=cast(int, row["max_count"]),
            sum=cast(int, row["sum"]),
        )
        for _, row in summary.iterrows()
    ]

    return summaries


In [92]:
relation_table = getO2OWithTypes(ocel)

summaries = summarize_relation_counts(
    relation_table=ocel.relations,
    direction="source",
    qualifier_col = ocel.qualifier,
    source_type_col = ocel.event_activity,
    target_type_col = ocel.object_type_column,
    source_id_col=ocel.event_id_column,
    target_id_col= ocel.object_id_column
    
)
summaries

[RelationCountSummary(qualifier='CR brought to bay', source='Bring to Loading Bay', target='Container', min_count=1, max_count=1, sum=1969),
 RelationCountSummary(qualifier='CR departed', source='Depart', target='Container', min_count=2, max_count=39, sum=1956),
 RelationCountSummary(qualifier='CR laded', source='Load Truck', target='Container', min_count=1, max_count=1, sum=10553),
 RelationCountSummary(qualifier='CR loaded', source='Load to Vehicle', target='Container', min_count=1, max_count=1, sum=1966),
 RelationCountSummary(qualifier='CR moved', source='Drive to Terminal', target='Container', min_count=1, max_count=1, sum=1989),
 RelationCountSummary(qualifier='CR picked', source='Pick Up Empty Container', target='Container', min_count=1, max_count=1, sum=1995),
 RelationCountSummary(qualifier='CR rescheduled', source='Reschedule Container', target='Container', min_count=1, max_count=1, sum=36),
 RelationCountSummary(qualifier='CR stored', source='Place in Stock', target='Contain

In [94]:
relation_table = getO2OWithTypes(ocel)

summaries = summarize_relation_counts(
    relation_table=relation_table,
    qualifier_col = "qualifier",
    source_type_col = "source_type",
    target_type_col = "target_type",
    source_id_col = "source",
    target_id_col = "target",
    direction = "target"
    
)
summaries

[RelationCountSummary(qualifier='CR for TD', source='Container', target='Transport Document', min_count=1, max_count=5, sum=1999),
 RelationCountSummary(qualifier='Ersatz VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=6, sum=33),
 RelationCountSummary(qualifier='High-Prio VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=5, sum=179),
 RelationCountSummary(qualifier='Regular VH for TD', source='Transport Document', target='Vehicle', min_count=1, max_count=10, sum=565),
 RelationCountSummary(qualifier='TD for CO', source='Customer Order', target='Transport Document', min_count=1, max_count=1, sum=594)]